In [7]:
from brian2 import *
import numpy as np

start_scope()

defaultclock.dt = 0.0001*ms  

# Custom timing function
@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, result=1, sum=1, spikes_received=1)
def spike_timing(w, global_clock, layer, sum, spikes_received): 
    #print(global_clock)
    x = global_clock % 1
    if w >= 0:
        return (x ** (1 - w)) 
    else:
        return (1 - (1 - x) ** (1 + w)) 
    
@implementation('numpy', discard_units=True)
@check_units(layer=1, result=1, sum=1, spikes_received=1)
def math1(layer, sum, spikes_received): 
    return (sum/spikes_received )+ layer


In [61]:
# Urd, Verdande, Skuld 

def run_Urd(inputs, weights_1, weights_2, weights_3):
    '''4-10-3 SNN'''
    # will add check of weights # so it all works
    n_input = 4 
    n_hidden = 10
    n_output = 3
    n_total = n_input + n_hidden + n_output

    neurons = NeuronGroup(n_total, '''
        v : 1
        sum : 1
        spikes_received : 1
        scheduled_time : second
        global_clock : 1
    ''', threshold='v > 1', reset='v = 0', method='exact')

    neurons.v = 0
    neurons.scheduled_time = 1e9 * second
    neurons.global_clock = 0.0
    neurons.sum = 0.0
    neurons.spikes_received = 0.0


    indices = list(range(n_input))
    stim = SpikeGeneratorGroup(n_input, indices=indices, times=(inputs*ms))

    syn_input = Synapses(stim, neurons[0:n_input], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    syn_input.connect(j='i')
    syn_input.w = weights_1
    syn_input.layer = 0

    syn_hidden = Synapses(neurons[0:n_input], neurons[n_input:n_input+n_hidden], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    syn_hidden.connect()
    syn_hidden.w = weights_2
    syn_hidden.layer = 1

    syn_output = Synapses(neurons[n_input:n_input+n_hidden], neurons[n_input+n_hidden:n_total], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    syn_output.connect()
    syn_output.w = weights_3
    syn_output.layer = 2

    neurons.run_regularly('''
        v = int(abs(t - scheduled_time) < 0.0005*ms) * 1.2
        global_clock += 0.001
    ''', dt=0.001*ms)


    spikemon = SpikeMonitor(neurons)

    run(5*ms)

    out_1 = 0
    out_2 = 0
    out_3 = 0

    #print("--- New Input Run Results ---")
    for i in range(n_total):
        times = spikemon.spike_trains()[i]
        #formatted = [f"{t/ms:.3f} ms" for t in times]
        #print(f"Neuron {i} spike times: {formatted}")
        if i == 14:
            out_1 = round(times[0]/ms - 2, 3)
        if i == 15:
            out_2 = round(times[0]/ms - 2, 3)
        if i == 16:
            out_3 = round(times[0]/ms - 2, 3)
    
    return out_1, out_2, out_3



In [68]:
inputs = np.random.uniform(0.001, .9, size=4) #[0.5]* 4
w_1 = np.random.uniform(0.01, .95, size=4) #[0.5] * 4
w_2 = np.random.uniform(0.01, .95, size=40) #[0.5] * 40
w_3 = np.random.uniform(0.01, .95, size=30) #[0.5] * 30



var = run_Urd(inputs, w_1, w_2, w_3)

print("results: ", var)




WARNING    Came across an abstract code block that may not be well-defined: the outcome may depend on the order of execution. You can ignore this warning if you are sure that the order of operations does not matter. 3 lines of abstract code, first line is: 'spikes_received += 1 (in-place)'
 [brian2.codegen.generators.base]
WARNING    Came across an abstract code block that may not be well-defined: the outcome may depend on the order of execution. You can ignore this warning if you are sure that the order of operations does not matter. 3 lines of abstract code, first line is: 'spikes_received += 1 (in-place)'
 [brian2.codegen.generators.base]
WARNING    Came across an abstract code block that may not be well-defined: the outcome may depend on the order of execution. You can ignore this warning if you are sure that the order of operations does not matter. 3 lines of abstract code, first line is: 'spikes_received += 1 (in-place)'
 [brian2.codegen.generators.base]


results:  (0.825, 0.836, 0.827)


In [2]:
list(range(3))


[0, 1, 2]